In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import time
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.config import RunnableConfig
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

In [4]:

class AgentState(TypedDict):
    question: str
    answer: str

In [5]:
def call_model(state: AgentState, config: RunnableConfig) -> AgentState:
    model_type = config["configurable"].get("model_type", "openai")
    if model_type == "ollama":
        print("Using Ollama (deepseek-r1:7b).")
        llm = ChatOllama(model="deepseek-r1", temperature=0)
    else:
        print("Using (Gemma2-9b-It).")
        llm =ChatGroq(groq_api_key="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",model_name="llama-3.1-8b-instant")

    messages = [HumanMessage(content=state["question"])]
    response = llm.invoke(messages)
    state["answer"] = response.content
    return state

In [6]:
workflow = StateGraph(AgentState)
workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)
graph = workflow.compile()
ollama_config = {
    "configurable": {
        "model_type": "ollama"
    }
}
graph.invoke({"question": "What's the highest mountain in the world?"}, ollama_config)

Using Ollama (deepseek-r1:7b).


{'question': "What's the highest mountain in the world?",
 'answer': '<think>\n\n</think>\n\nThe highest mountain in the world is Mount Everest.'}